In [1]:
import pandas as pd
import numpy as np
import math

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import sys
from pathlib import Path
import torch
sys.path.insert(0, "..")
from src.data_utils import compute_cluster_medians

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
path = "../output/data/20251108_tucker_2014_January_top_53_store_2000_item_growth_rate_clusters.pt"
model_dict = torch.load(path, map_location='cpu')

In [4]:
 model_dict["assignments"]["factor_name"].unique()

array(['Store', 'SKU', 'Feature'], dtype=object)

In [11]:
store_assignments = model_dict["assignments"].query("factor_name == 'Store'")
store_assignments.rename(columns={"item_name": "store"}, inplace=True)
store_assignments["store"] = store_assignments["store"].astype(int)
store_assignments.head()

/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_87846/4029036206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_assignments.rename(columns={"item_name": "store"}, inplace=True)
/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_87846/4029036206.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_assignments["store"] = store_assignments["store"].astype(int)


,factor_name,store,cluster_id
0,Store,1,2
1,Store,1,6
2,Store,1,4
3,Store,1,8
4,Store,1,5


In [5]:
store_assignments["factor_name"].unique()

array(['Store'], dtype=object)

In [6]:
store_assignments["cluster_id"].unique()

array([ 2,  6,  4,  8,  5,  3, 13, 20,  0, 16,  7, 14, 19, 22, 26, 11, 23,
       29, 24, 21, 34, 10, 17, 36, 39, 35,  1, 32, 31, 25, 18, 12, 28, 37,
       30,  9, 15, 27, 38, 33])

In [7]:
path= "../output/data/2013_2014_store_2000_item_cyc_features.parquet"
df = pd.read_parquet(path)
df.head()

,date,store_item,store,item,unit_sales,onpromotion,transactions,cluster,type_B,type_C,...,monthofyear_sin,monthofyear_cos,paycycle_sin,paycycle_cos,season_sin,season_cos,unit_sales_rolling_median,unit_sales_ewm_decay,growth_rate_rolling_median,growth_rate_ewm_decay
453,2013-01-02,10_1003679,10,1003679,2.0,None,1293.0,15,0,1,...,0.5,0.866025,0.743145,0.669131,-0.970064,0.242850,2.0,2.0,0.000000,0.000000
32621,2013-01-03,10_1003679,10,1003679,5.0,None,1157.0,15,0,1,...,0.5,0.866025,0.951057,0.309017,-0.965740,0.259512,5.0,5.0,1.500000,1.500000
63664,2013-01-04,10_1003679,10,1003679,2.0,None,970.0,15,0,1,...,0.5,0.866025,0.994522,-0.104529,-0.961130,0.276097,2.0,2.0,-0.600000,-0.600000
94751,2013-01-05,10_1003679,10,1003679,6.0,None,1269.0,15,0,1,...,0.5,0.866025,0.866025,-0.500000,-0.956235,0.292600,6.0,6.0,2.000000,2.000000
127623,2013-01-06,10_1003679,10,1003679,4.0,None,1173.0,15,0,1,...,0.5,0.866025,0.587785,-0.809017,-0.951056,0.309017,4.0,4.0,-0.333333,-0.333333


In [8]:
df["date"] = pd.to_datetime(df["date"])
df.sort_values(["date", "store_item"], inplace=True, kind="mergesort")

In [13]:
df = df.merge(store_assignments[["store", "cluster_id"]], on=["store"], how="left")

: 

In [ ]:
compute_cluster_medians(df, date_col="date", cluster_col="cluster_id", value_col="unit_sales")